<a href="https://colab.research.google.com/github/simulate111/Introduction-to-Human-Language-Technology/blob/main/ex6_solved_imdb_parameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solution to exercise 6:

* The code is based on the lecture notebook, with mostly GPT4-generated code for parameter grid search at the end :)

In [1]:
# add optuna and plotly since GPT4 thought these are good to use :)
!pip3 install -q transformers[torch] datasets evaluate optuna plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
from pprint import pprint #pprint => pretty-print, I use it occassionally throughout the notebook
import datasets
dset=datasets.load_dataset("imdb")
pprint(dset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [3]:
dset=dset.shuffle() #This is never a bad idea, datasets may have ordering to them, which is not what we want
del dset["unsupervised"] #Delete the unlabeled part of the dataset, we don't need it for anything

In [4]:
import sklearn.feature_extraction

vectorizer=sklearn.feature_extraction.text.CountVectorizer(binary=True,max_features=20000)

texts=[ex["text"] for ex in dset["train"]] #get a list of all texts from the training data
vectorizer.fit(texts) #"Trains" the vectorizer, i.e. builds its vocabulary


CountVectorizer(binary=True, max_features=20000)

In [5]:
def vectorize_example(ex):
    vectorized=vectorizer.transform([ex["text"]]) # [...] because the vectorizer expects a list/iterable over inputs, not one input
    non_zero_features=vectorized.nonzero()[1] #.nonzero gives a pair of (rows,columns), we want the columns
    non_zero_features+=1 #feature index 0 will have a special meaning
                         # so let us not produce it by adding +1 to everything
    return {"input_ids":non_zero_features}

vectorized=vectorize_example(dset["train"][0])

In [6]:
# We can map back to vocabulary and check that everything works
# vectorizer.vocabulary_ is a dictionary {key:word, value:idx}

idx2word=dict((i,w) for (w,i) in vectorizer.vocabulary_.items()) #inverse the vocab dictionary
words=[]
for idx in vectorized["input_ids"]:
    words.append(idx2word[idx-1]) ## It is easy to forgot we moved all by +1
pprint(", ".join(words)) #This is now the bag of words representation of the document

('140, 33, about, across, all, altman, always, an, and, appear, are, aren, '
 'art, as, associated, at, available, bag, based, be, been, being, best, '
 'better, between, bits, blame, board, bombed, box, br, but, by, can, car, '
 'case, cast, character, characters, color, come, course, crime, criterion, '
 'did, different, don, down, dramas, drives, due, dump, edge, effectively, '
 'emotions, error, especially, ever, every, failed, father, film, filmic, '
 'filmmaker, filmmakers, films, finally, find, first, followed, for, foremost, '
 'four, from, full, garbage, going, great, group, hang, happy, has, he, '
 'higher, him, himself, his, history, hit, holding, homes, hour, if, '
 'imaginary, in, intriguing, is, isn, it, its, last, laughter, least, like, '
 'linked, listed, live, lives, living, long, made, magical, makes, man, '
 'master, material, means, minute, misfire, mix, mixed, moments, most, movie, '
 'much, multi, night, normal, not, occasionally, of, often, on, one, only, or, '
 

In [7]:
# Apply the tokenizer to the whole dataset using .map()
dset_tokenized = dset.map(vectorize_example,num_proc=4)
pprint(dset_tokenized["train"][0])

Map (num_proc=4):   0%|          | 0/25000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/25000 [00:00<?, ? examples/s]

{'input_ids': [26,
               173,
               309,
               416,
               727,
               787,
               793,
               860,
               887,
               1039,
               1115,
               1118,
               1182,
               1207,
               1270,
               1300,
               1421,
               1529,
               1652,
               1702,
               1756,
               1793,
               1880,
               1897,
               1901,
               1988,
               2017,
               2121,
               2169,
               2279,
               2295,
               2604,
               2625,
               2711,
               2784,
               2864,
               2878,
               3053,
               3062,
               3560,
               3591,
               4159,
               4298,
               4316,
               5089,
               5104,
               5429,
               5490,
  

In [8]:
import torch

def collator(list_of_examples):
    batch={"labels":torch.tensor(list(ex["label"] for ex in list_of_examples))} #this is easy, labels are made into a single tensor
    #the worse bit is now to pad the examples, as they are of different length
    tensors=[]
    max_len=max(len(example["input_ids"]) for example in list_of_examples) #this is the longest example in the batch
    #everything needs to be padded to fit in length the longest example
    #(so we can build a single tensor out of it)
    for example in list_of_examples:
        ids=torch.tensor(example["input_ids"]) #pick the input ids
        # pad(what,(from_left, from_right)) <- this is how we call the stock pad function
        padded=torch.nn.functional.pad(ids,(0,max_len-ids.shape[0])) #pad by max - current length, pads with zero by default
        tensors.append(padded) #accumulated the padded ids
    batch["input_ids"]=torch.vstack(tensors) #now that we have all of them the same length, a simple vstack() stacks them up
    return batch #...and that's all there is to it


# The first changes happen below:
1. We can make the embedding dimension 1, i.e. the embeddings will be vectors of length 1, i.e. just individual real numbers. We do not need to change the code of the model, we can just change the number in model config.
2. Just for clarity and clean approach, let us remove the `tanh` nonlinearity from the network, so the decision becomes a simple sum of weights. (This does not affect the outcome really, since `tanh` is monotonic and thus preserves order)

In [9]:
import torch
import transformers

# A model wants a config, I can simply inherit from the base
# class for pretrained configs
class MLPConfig(transformers.PretrainedConfig):
    pass

# This is the model
class MLP(transformers.PreTrainedModel):

    config_class=MLPConfig

    # In the initialization method, one instantiates the layers
    # these will be, for the most part the trained parameters of the model
    def __init__(self,config):
        super().__init__(config)
        self.vocab_size=config.vocab_size #embedding matrix row count
        # Build and initialize embedding of vocab size +1 x hidden size (+1 because of the padding index 0!)
        self.embedding=torch.nn.Embedding(num_embeddings=self.vocab_size+1,embedding_dim=config.hidden_size,padding_idx=0)
        # Normally you would not initialize these yourself, but I have my reasons here ;)
        torch.nn.init.uniform_(self.embedding.weight.data,-0.001,0.001) #initialize the embeddings with small random values
        # Note! This is quite clever and keeps the embedding for 0, the padding, pure zeros
        # This takes care of the lower half of the network, now the upper half
        # Output layer: hidden size x output size
        self.output=torch.nn.Linear(in_features=config.hidden_size,out_features=config.nlabels)
        # Now we have the parameters of the model


    # The computation of the model is put into the forward() function
    # it receives a batch of data and optionally the correct `labels`
    #
    # If given `labels` it returns (loss,output)
    # if not, then it returns (output,)
    def forward(self,input_ids,labels=None): #nevermind the attention_mask, its time will come, data collator insists on adding it
        #1) sum up the embeddings of the items
        embedded=self.embedding(input_ids) #(batch,ids)->(batch,ids,embedding_dim)
        # Since the Embedding keeps the first row of the matrix pure zeros, we don't need to worry about the padding
        # so next we sum the embeddings across the word dimension
        # (batch,ids,embedding_dim) -> (batch,embedding_dim)
        embedded_summed=torch.sum(embedded,dim=1)

        #2) apply non-linearity
        # (batch,embedding_dim) -> (batch,embedding_dim)

        #### MODIFIED HERE FOR EXERCISE 5 -> commented out
        ####projected=torch.tanh(embedded_summed) #Note how non-linearity is applied here and not when configuring the layer in __init__()

        #3) and now apply the upper, output layer of the network
        # (batch,embedding_dim) -> (batch, num_of_classes i.e. 2 in our case)

        #### MODIFIED HERE FOR EXERCISE 5 -> base it off embedded_summed
        ##### OLD: logits=self.output(projected)
        logits=self.output(embedded_summed)

        # ...and that's all there is to it!

        #print("input_ids.shape",input_ids.shape)
        #print("embedded.shape",embedded.shape)
        #print("embedded_summed.shape",embedded_summed.shape)
        #print("projected.shape",projected.shape)
        #print("logits.shape",logits.shape)

        # We have labels, so we ought to calculate the loss
        if labels is not None:
            loss=torch.nn.CrossEntropyLoss() #This loss is meant for classification, so let's use it
            # You run it as loss(model_output,correct_labels)
            return (loss(logits,labels),logits)
        else:
            # No labels, so just return the logits
            return (logits,)

# Configure the model:
#   these parameters are used in the model's __init__()


####### CHANGED HERE FOR EXERCISE 5, hidden_size from 20 to 1
mlp_config=MLPConfig(vocab_size=len(vectorizer.vocabulary_),hidden_size=1,nlabels=2)


In [10]:
# And we can make a model
mlp=MLP(mlp_config)
fake_batch=collator([dset_tokenized["train"][0],dset_tokenized["train"][1]])
mlp(**fake_batch) #** expands input_ids and labels as parameters of the call

(tensor(0.6526, grad_fn=<NllLossBackward0>),
 tensor([[0.8306, 0.8874],
         [0.7847, 0.8940]], grad_fn=<AddmmBackward0>))

In [11]:
# Set training arguments
# their names are mostly self-explanatory
trainer_args = transformers.TrainingArguments(
    "mlp_checkpoints", #save checkpoints here
    evaluation_strategy="steps",
    logging_strategy="steps",
    eval_steps=500,
    logging_steps=500,
    learning_rate=1e-4, #learning rate of the gradient descent
    max_steps=20000,
    load_best_model_at_end=True,
    per_device_train_batch_size=128
)

pprint(trainer_args)

TrainingArguments(
_n_gpu=0,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=500,
evaluation_strategy=steps,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_l

In [12]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_accuracy(outputs_and_labels):
    outputs, labels = outputs_and_labels
    predictions = np.argmax(outputs, axis=-1) #pick the index of the "winning" label
    return accuracy.compute(predictions=predictions, references=labels)

In [13]:
# Make a new model
mlp = MLP(mlp_config)


# Argument gives the number of steps of patience before early stopping
# i.e. training is stopped when the evaluation loss fails to improve
# certain number of times
early_stopping = transformers.EarlyStoppingCallback(5)

trainer = transformers.Trainer(
    model=mlp,
    args=trainer_args,
    train_dataset=dset_tokenized["train"],
    eval_dataset=dset_tokenized["test"].select(range(1000)), #make a smaller subset to evaluate on
    compute_metrics=compute_accuracy,
    data_collator=collator,
    callbacks=[early_stopping]
)

# FINALLY!
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,0.744200,0.678311,0.558000
1000,0.644500,0.616397,0.674000
1500,0.569200,0.554005,0.776000
2000,0.496700,0.500924,0.809000
2500,0.437500,0.459545,0.828000
3000,0.391000,0.427146,0.841000
3500,0.354500,0.401557,0.849000
4000,0.322800,0.381076,0.855000
4500,0.300300,0.364905,0.856000
5000,0.280400,0.352063,0.857000


TrainOutput(global_step=13500, training_loss=0.29045999032479747, metrics={'train_runtime': 388.7591, 'train_samples_per_second': 6585.055, 'train_steps_per_second': 51.446, 'total_flos': 17644197888.0, 'train_loss': 0.29045999032479747, 'epoch': 68.88})

We can then evaluate the trained model on a given dataset (here our test subset) by calling [`Trainer.evaluate`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer.evaluate):

In [14]:
eval_results = trainer.evaluate(dset_tokenized["test"])

print(eval_results)

{'eval_loss': 0.2778298854827881, 'eval_accuracy': 0.88852, 'eval_runtime': 8.9254, 'eval_samples_per_second': 2801.01, 'eval_steps_per_second': 350.126, 'epoch': 68.88}


# Now we can try GPT4's solution:

* It recommends to use a library called `optuna`
* It falsely claims there is no parameter search support in the HF Trainer API
* It generally produces valid code which can be quite quickly modified to produce the desired parameter search
* Probably with some extra effort, it could have written more of it
* The chat log is at the end of this notebook

In [15]:
import optuna

def objective(trial):
    # Define the search space for hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 64, 128, 256])


    trainer_args = transformers.TrainingArguments(
        "mlp_checkpoints", #save checkpoints here
        evaluation_strategy="steps",
        logging_strategy="steps",
        eval_steps=500,
        logging_steps=500,
        learning_rate=learning_rate, #learning rate of the gradient descent
        max_steps=10000, #I will keep this small, no time to wait too long :)
        load_best_model_at_end=True,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size
    )

    mlp = MLP(mlp_config)

    trainer = transformers.Trainer(
        model=mlp,
        args=trainer_args,
        train_dataset=dset_tokenized["train"],
        eval_dataset=dset_tokenized["test"].select(range(1000)), #make a smaller subset to evaluate on
        compute_metrics=compute_accuracy,
        data_collator=collator,
        callbacks=[early_stopping]
    )

    # Train the model and get the best validation loss
    trainer.train()
    eval_results = trainer.evaluate()
    return eval_results["eval_accuracy"] #let's try to maximize accuracy

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=7) #I will keep this small, no time to wait :)

[I 2024-04-06 08:30:12,951] A new study created in memory with name: no-name-9efef484-6ec9-40aa-a4ca-5c641ba7387d
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,0.629900,0.579448,0.794000
1000,0.511900,0.502250,0.832000
1500,0.434100,0.450916,0.844000
2000,0.379600,0.415139,0.853000
2500,0.340500,0.389473,0.857000
3000,0.311400,0.370627,0.863000
3500,0.289100,0.356380,0.865000
4000,0.269000,0.345555,0.866000
4500,0.256200,0.337401,0.868000
5000,0.243900,0.331194,0.868000


Checkpoint destination directory mlp_checkpoints/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory mlp_checkpoints/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory mlp_checkpoints/checkpoint-1500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory mlp_checkpoints/checkpoint-2000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory mlp_checkpoints/checkpoint-2500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory mlp_checkpoints/checkpoint-3000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory mlp_checkpoints/checkpoint-3500 already exists and is no

[I 2024-04-06 08:34:55,229] Trial 0 finished with value: 0.872 and parameters: {'learning_rate': 0.00010288584526058033, 'batch_size': 128}. Best is trial 0 with value: 0.872.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,0.789600,0.761869,0.510000
1000,0.762400,0.735261,0.510000
1500,0.724800,0.719903,0.512000
2000,0.712700,0.708860,0.515000
2500,0.701700,0.701252,0.511000
3000,0.695300,0.695434,0.519000
3500,0.688300,0.690788,0.528000
4000,0.684800,0.687176,0.530000
4500,0.678600,0.684014,0.537000
5000,0.675700,0.681284,0.538000


...

Checkpoint destination directory mlp_checkpoints/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory mlp_checkpoints/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory mlp_checkpoints/checkpoint-1500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory mlp_checkpoints/checkpoint-2000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory mlp_checkpoints/checkpoint-2500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory mlp_checkpoints/checkpoint-3000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory mlp_checkpoints/checkpoint-3500 already exists and is no

[I 2024-04-06 09:00:44,383] Trial 5 finished with value: 0.668 and parameters: {'learning_rate': 4.374107389740306e-06, 'batch_size': 256}. Best is trial 0 with value: 0.872.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,0.690700,0.610712,0.658000
1000,0.544300,0.524988,0.782000
1500,0.458700,0.470475,0.818000
2000,0.400600,0.432376,0.835000
2500,0.358500,0.404446,0.845000
3000,0.327000,0.383515,0.854000
3500,0.302800,0.367558,0.857000
4000,0.281300,0.355281,0.859000
4500,0.267400,0.345926,0.861000
5000,0.254400,0.338746,0.863000


Checkpoint destination directory mlp_checkpoints/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory mlp_checkpoints/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory mlp_checkpoints/checkpoint-1500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory mlp_checkpoints/checkpoint-2000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory mlp_checkpoints/checkpoint-2500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory mlp_checkpoints/checkpoint-3000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory mlp_checkpoints/checkpoint-3500 already exists and is no

[I 2024-04-06 09:05:24,462] Trial 6 finished with value: 0.871 and parameters: {'learning_rate': 0.00010078286464421923, 'batch_size': 128}. Best is trial 0 with value: 0.872.


In [16]:
# Print the best trial's hyperparameters and objective value
best_trial = study.best_trial
print(f"Best trial (number {best_trial.number}):")
print(f"  Value: {best_trial.value}")
print(f"  Params: {best_trial.params}")

# Print all trials' hyperparameters and objective values
print("\nAll trials:")
for trial in study.trials:
    print(f"  Trial {trial.number}:")
    print(f"    Value: {trial.value}")
    print(f"    Params: {trial.params}")

Best trial (number 0):
  Value: 0.872
  Params: {'learning_rate': 0.00010288584526058033, 'batch_size': 128}

All trials:
  Trial 0:
    Value: 0.872
    Params: {'learning_rate': 0.00010288584526058033, 'batch_size': 128}
  Trial 1:
    Value: 0.578
    Params: {'learning_rate': 5.534229599224339e-06, 'batch_size': 16}
  Trial 2:
    Value: 0.508
    Params: {'learning_rate': 6.490536326581467e-06, 'batch_size': 64}
  Trial 3:
    Value: 0.51
    Params: {'learning_rate': 1.0079750570117843e-06, 'batch_size': 256}
  Trial 4:
    Value: 0.536
    Params: {'learning_rate': 2.8669105140322417e-06, 'batch_size': 128}
  Trial 5:
    Value: 0.668
    Params: {'learning_rate': 4.374107389740306e-06, 'batch_size': 256}
  Trial 6:
    Value: 0.871
    Params: {'learning_rate': 0.00010078286464421923, 'batch_size': 128}


In [17]:
import optuna.visualization as vis
fig = vis.plot_parallel_coordinate(study)
fig.show()

In [18]:
fig = vis.plot_slice(study)
fig.show()

In [19]:
fig = vis.plot_contour(study, params=['learning_rate', 'batch_size'])
fig.show()

In [20]:
fig = vis.plot_optimization_history(study)
fig.show()

**Here is my chat log with GPT4 grabbing the solution for the exercise.**

What is the preferred way to do grid search for hyperparameters in HuggingFace trainer? Is there a nice, simple API for that?
--------------------------
HuggingFace does not provide a built-in grid search functionality in its Trainer API. However, you can easily integrate other libraries such as Scikit-Learn's GridSearchCV or Optuna to perform hyperparameter tuning with the HuggingFace library.

Here's an example of using Optuna for hyperparameter tuning with the HuggingFace Trainer:

First, install Optuna:
Copy code
`pip install optuna`

Create a function to set up the HuggingFace Trainer and define the objective for optimization. In this case, we will be minimizing the validation loss.
python
Copy code

```
import optuna
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

def objective(trial):
    # Define the search space for hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-6, 1e-3, log=True)
    num_train_epochs = trial.suggest_int("num_train_epochs", 1, 5)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])

    # Load the dataset, tokenizer, and model
    dataset = ...
    tokenizer = AutoTokenizer.from_pretrained("your-pretrained-model")
    model = AutoModelForSequenceClassification.from_pretrained("your-pretrained-model")

    # Set up training arguments
    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=num_train_epochs,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        eval_steps=100,
        disable_tqdm=True,
        report_to="none",
    )

    # Set up the trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
    )

    # Train the model and get the best validation loss
    trainer.train()
    eval_results = trainer.evaluate()
    return eval_results["eval_loss"]
```


Run the hyperparameter search using Optuna.
python
Copy code

```
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)
```

You can adjust the search space and the number of trials depending on your needs. Optuna will run the training with different combinations of hyperparameters and will keep track of the best ones based on the validation loss. You can also customize the objective function to optimize other metrics if needed.
--------------------------
Once I have the optimized `study` object, is there a way to print the trials w.r.t. individual parameters and get insight into their effect?
--------------------------
Yes, you can analyze the trials in the study object to gain insights into the effects of individual parameters. Optuna provides various visualization functions that can help you understand the optimization process and the relationships between hyperparameters.

Here's an example of how to use Optuna's visualization functions:

First, install the required packages:
bash
Copy code

```
pip install optuna
pip install plotly
```
Import the necessary modules:
python
Copy code

```
import optuna.visualization as vis
```
Use visualization functions to analyze the trials:
Plot a parallel coordinate plot to visualize the relationship between hyperparameters and the objective value:
python
Copy code

```
fig = vis.plot_parallel_coordinate(study)
fig.show()
```

Plot a slice plot to visualize the relationship between individual hyperparameters and the objective value:
python
Copy code

```
fig = vis.plot_slice(study)
fig.show()
```

Plot a contour plot to visualize the relationship between pairs of hyperparameters and the objective value:
python
Copy code

```
fig = vis.plot_contour(study, params=['learning_rate', 'num_train_epochs', 'batch_size'])
fig.show()
```

Plot an optimization history to visualize the progress of the optimization process:
python
Copy code

```
fig = vis.plot_optimization_history(study)
fig.show()
```

These visualizations can help you understand the effects of individual parameters on the objective value, as well as the relationships between parameters.

Additionally, you can print a summary of the study's trials:

python
Copy code

```
# Print the best trial's hyperparameters and objective value
best_trial = study.best_trial
print(f"Best trial (number {best_trial.number}):")
print(f"  Value: {best_trial.value}")
print(f"  Params: {best_trial.params}")

# Print all trials' hyperparameters and objective values
print("\nAll trials:")
for trial in study.trials:
    print(f"  Trial {trial.number}:")
    print(f"    Value: {trial.value}")
    print(f"    Params: {trial.params}")
```

This will give you an overview of all the trials conducted during the hyperparameter optimization and their respective results.